# Test Assessor Generalization w.r.t. distribution shift

We select the best/worst performing tasks for the training set, en the reverse for the test set.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
from pprint import pprint

import bigbench.api.results as bb
import numpy as np

from lass.log_handling import LogLoader, TaskLog
from lass.datasets import analyse, merge, huggingfaceify, to_dataframe

In [3]:
loader = (LogLoader(logdir = Path('../artifacts/logs'))
        .with_tasks('paper-full')
        .with_model_families(['BIG-G T=0'])
        .with_model_sizes(['128b'])
        .with_shots([0])
        .with_query_types([bb.MultipleChoiceQuery])
)

data = to_dataframe(loader)

Split into good/bad performing training set, and reverse test set.

In [4]:

accs = (data
    .groupby('task', as_index=False).agg(acc=('correct', 'mean')) # type: ignore
    .sort_values('acc', ascending=False))

train_tasks, test_tasks = np.split(accs['task'], [int(len(accs) * 0.8)])
len(train_tasks), len(test_tasks)

train = data[data['task'].isin(train_tasks)]
test = data[data['task'].isin(test_tasks)]

stats = merge(analyse(train), analyse(test), 'train', 'test')
pprint(stats)
train.head(1)

{'metrics': {'conf-absolute': {'roc_auc': {'test': 0.48721947856416836,
                                           'train': 0.496735045546435}},
             'conf-normalized': {'roc_auc': {'test': 0.4934099804955599,
                                             'train': 0.5874931359228416}},
             'task-acc': {'test': 0.1310840005083238,
                          'train': 0.4182858596134283}},
 'stats': {'n_instances': {'test': 15738, 'train': 39320},
           'n_instances_nonbinary': {'test': 0, 'train': 373},
           'n_tasks': {'test': 24, 'train': 94}}}


,input,targets,scores,target_values,correct,absolute_scores,normalized_scores,metrics,task,shots
0,"In what follows, we provide short narratives, ...","[Seek and you shall find, It's better to light...","[-17.807415008544922, -18.298959732055664, -12...","{'April showers bring forth May flowers': 1, '...",0.0,"[-17.807415008544922, -18.298959732055664, -12...","[-5.244009971618652, -5.7355546951293945, -0.0...",{'calibration_multiple_choice_brier_score': 0....,abstract_narrative_understanding,0


In [5]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import torch

print(torch.cuda.device_count())

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


1


In [6]:
dataset = huggingfaceify(train, test)
dataset['train'][0]

{'text': "In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: Carla was having trouble juggling college, working at the diner and being a mother. She never had a day off and was burnt out. Her friend told her that her hard work would pay off one day and to keep going! Carla's friend was right; after a few more years of hard work, Carla graduated school and was able to get a better job. She was able to take a vacation and become overall successful.\nThis narrative is a good illustration of the following proverb: ",
 'label': 0}

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024) # longformer
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="np") #gpt-2
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=2048) # xlnet

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7efcb9b09240> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

In [8]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42) #.select(range(50))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42) #.select(range(50))
len(train_dataset), len(eval_dataset)

(39320, 15738)

In [9]:
import wandb
%env WANDB_PROJECT=lass
%env WANDB_LOG_MODEL=true
%env TOKENIZERS_PARALLELISM=true
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: WANDB_PROJECT=lass
env: WANDB_LOG_MODEL=true
env: TOKENIZERS_PARALLELISM=true


wandb: Currently logged in as: wschella. Use `wandb login --relogin` to force relogin


True

In [10]:
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import BertModel, BertConfig
import numpy as np
import scipy
import torch
import lass, lass.metrics.hf

# model = BertModel(BertConfig.from_pretrained("bert-base-cased"))
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2)
# model.config.pad_token_id = model.config.eos_token_id
# model = AutoModelForSequenceClassification.from_pretrained("./test_trainer/checkpoint-13500", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("../artifacts/assessors/bert-bs32/checkpoint-3000", num_labels=2)

training_args = TrainingArguments(
    output_dir="albert-bs32-0sh-task-split-DS-ls2",
    run_name="albert-bs32-0sh-task-split-DS-ls2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="wandb",
    num_train_epochs=10,
    # weight_decay=0.05,
    label_smoothing_factor=0.2,
)

compute_metrics = lass.metrics.hf.get_metric_computer([
  "accuracy",
  "precision",
  "recall",
  "f1",
  "roc_auc",
  "brier_score",
])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
trainer.train()

wandb.finish()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.


/home/wout/pp/lass/.env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 39320


  Num Epochs = 10


  Instantaneous batch size per device = 32


...

  Batch size = 32


/home/wout/pp/lass/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Saving model checkpoint to albert-bs32-0sh-task-split-DS-ls2/checkpoint-1500


Configuration saved in albert-bs32-0sh-task-split-DS-ls2/checkpoint-1500/config.json


Model weights saved in albert-bs32-0sh-task-split-DS-ls2/checkpoint-1500/pytorch_model.bin
